Open Points:
1. Find out until what date 
2. How to query more than 6000 data points from the graph
3. Insert a column that creates the symbol ticket dai/weth and weth/dai using columns tokenin.symbol and tokenout.symbol -> first check the dune

In [84]:
import requests
import pandas as pd
import numpy as np


#maximum items to query at once is 1000
#implement a skip variable to loop over all transactions returning 1000 transactions for each iteration. -> While loop
def get_messarisubg_uniswap_swaps(skip =0 ,limit=1000):

    url = "https://api.thegraph.com/subgraphs/name/messari/uniswap-v3-ethereum"

    query = """{
  swaps(skip: """ +str(skip)+""",first: """ +str(limit)+""", orderBy: timestamp, orderDirection: desc) {
    timestamp
    blockNumber
    hash
    from
    amountIn
    amountInUSD
    amountOut
    amountOutUSD
    tokenIn {
      id
      name
      symbol
      lastPriceUSD
      decimals
    }
    tokenOut {
      id
      name
      symbol
      lastPriceUSD
      decimals
    }
    pool {
      id
      name
      totalValueLockedUSD
      symbol
    }
  }}"""
    headers = {"Content-Type": "application/json"} #define in the request https header the content type: we want to send a json query in the body (there are a total of 4 types of http header types)
    #HTTP POST is a request method of an Hypertext Transfer Protocol (HTTP) and is used to create or update a resource on the server. The HTTP POST method is used to submit web forms and files and images to the server.
    response = requests.post(url, headers=headers, json={"query": query}) #define in the http request body the json data we want to send to the server

    return response.json()

#get more than first 1000 rows
skip_counter = 0
list_of_df = []
while True:
    try: 

        result = get_messarisubg_uniswap_swaps(skip = skip_counter)
        list_of_df.append(pd.json_normalize(result["data"]["swaps"]))
        skip_counter += 1000
        
        
    except Exception: 
      break



df_swaps = pd.concat(list_of_df).reset_index(drop = True)

df_swaps #gives me 6000 entries for the last 4 hours. MAx to query from thegraph at once is 6000 swaps. 



,timestamp,blockNumber,hash,from,amountIn,amountInUSD,amountOut,amountOutUSD,tokenIn.id,tokenIn.name,...,tokenIn.decimals,tokenOut.id,tokenOut.name,tokenOut.symbol,tokenOut.lastPriceUSD,tokenOut.decimals,pool.id,pool.name,pool.totalValueLockedUSD,pool.symbol
0,1676109827,16604802,0xf3ceca82382e1cf64bcee3a975cd64aefd073be4d41b...,0x32151c90cb424913d44b63ba46b23c9611a4fd63,9892931989243306022015660,123.9995173316189282322149819781634,81246916302049000,123.6267572650010656178502461329076,0x95ad61b0a150d79219dcf64e1e6cc01f0b64c4ce,SHIBA INU,...,18,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,Wrapped Ether,WETH,1521.617839690037204086788951788608,18,0x2f62f2b4c5fcd7570a709dec05d68ea19c82a9ec,Uniswap V3 SHIBA INU/Wrapped Ether 0.3%,3217593.34795739242540562593687591,SHIBA INU/Wrapped Ether
1,1676109827,16604802,0xd0d42dd251c528068c605e257d9c5017b691fb86e7a9...,0x758e8229dd38cf11fa9e7c0d5f790b4ca16b3b16,4276931285858154296,6507.854943690217706972962258183961,6507821724,6507.821724,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,Wrapped Ether,...,18,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USD Coin,USDC,1,6,0xe0554a476a092703abdb3ef35c80e0d76d32939f,Uniswap V3 USD Coin/Wrapped Ether 0.01%,25054146.87971317185871843637697096,USD Coin/Wrapped Ether
2,1676109827,16604802,0x69c88063f26c810aa69da8c4580b29b2287308915ab3...,0x758e8229dd38cf11fa9e7c0d5f790b4ca16b3b16,6798058235,6798.058235,4466749410830005163,6796.685588943898927288351338250403,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USD Coin,...,6,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,Wrapped Ether,WETH,1521.617839690037204086788951788608,18,0xe0554a476a092703abdb3ef35c80e0d76d32939f,Uniswap V3 USD Coin/Wrapped Ether 0.01%,25054146.87971317185871843637697096,USD Coin/Wrapped Ether
3,1676109827,16604802,0x365ae4ccad72eb42e20103908d794baf845e1ab178df...,0xd2d7cfcd0b4c6dc48a059b0b7dd5819075c36345,506379082,506.379082,332545464413587985,506.0071111597238944677594400559051,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USD Coin,...,6,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,Wrapped Ether,WETH,1521.617839690037204086788951788608,18,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,Uniswap V3 USD Coin/Wrapped Ether 0.05%,193380095.2313243878525520225914967,USD Coin/Wrapped Ether
4,1676109827,16604802,0x1ba071f07659474f0f5dbca138ba111f976c1154ae61...,0xc4c162e5e2475675cf07f4c851659d0f4266c224,14603651611,14603.651611,9590416496034505845,14592.94883042372103825323865071247,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USD Coin,...,6,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,Wrapped Ether,WETH,1521.617839690037204086788951788608,18,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,Uniswap V3 USD Coin/Wrapped Ether 0.05%,193380095.2313243878525520225914967,USD Coin/Wrapped Ether
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5995,1676097959,16603817,0x1f83fc001f13a802e5ae8742b586767f35d769dae02a...,0x315d2ee4fccda0def532ef4108ff57204f8d9eba,229481720619548576,348.5811252043791626176853999796588,35543238107999346269309,0,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,Wrapped Ether,...,18,0x9e46a38f5daabe8683e10793b06749eef7d733d1,Nectar,NCT,0,18,0x05aaa0053fa5c28e8c558d4c648cc129bea45018,Uniswap V3 Nectar/Wrapped Ether 1%,10931.96923997245243397716693465806,Nectar/Wrapped Ether
5996,1676097947,16603816,0xef2f29539477f93b536374b4640225f9d9d7c6589995...,0x18aee3221610c19c261f640b5a88f35c83f369d0,10924820007896433,16.59472502091502019320084492090963,75000000000,0,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,Wrapped Ether,...,18,0x3d658390460295fb963f54dc0899cfb1c30776df,CircuitsOfValue,Coval,0,8,0x2b43fe4f41d871fbc75af6e0ce85bce38ff1edc7,Uniswap V3 CircuitsOfValue/Wrapped Ether 1%,147235.1325041606973927920008979377,CircuitsOfValue/Wrapped Ether
5997,1676097947,16603816,0xc7d44761601d60b6b56409c6e94ed5d94eff0c35a7a2...,0xdaa822a52ac3a392e0619ce3c849f67d6a9482da,677947962543575638677,677.947962543575638677,677484820,677.48482,0x6b175474e89094c44da98b954eedeac495271d0f,Dai Stablecoin,...,

In [ ]:
# 

In [85]:
#a row can have multiple entries per transaction: in case more than one swap happened on uniswap
#example:transaction 0x1d8dc4cbb674f7a77437a8934aa841153dc52a538ce0c7765034e50103a83fda -> second entry is a token to token swap (i.e. FRAX for USDC)
df_swaps[df_swaps["hash"] == "0x1d8dc4cbb674f7a77437a8934aa841153dc52a538ce0c7765034e50103a83fda"]



,timestamp,blockNumber,hash,from,amountIn,amountInUSD,amountOut,amountOutUSD,tokenIn.id,tokenIn.name,...,tokenIn.decimals,tokenOut.id,tokenOut.name,tokenOut.symbol,tokenOut.lastPriceUSD,tokenOut.decimals,pool.id,pool.name,pool.totalValueLockedUSD,pool.symbol
612,1676108603,16604700,0x1d8dc4cbb674f7a77437a8934aa841153dc52a538ce0...,0x7fcc933f6042f840864886fa7f32e0397d3218eb,3188672750,3188.67275,2095504199301273606,3188.556572802205113147433639649221,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USD Coin,...,6,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,Wrapped Ether,WETH,1521.617839690037204086788951788608,18,0xe0554a476a092703abdb3ef35c80e0d76d32939f,Uniswap V3 USD Coin/Wrapped Ether 0.01%,25054146.87971317185871843637697096,USD Coin/Wrapped Ether
613,1676108603,16604700,0x1d8dc4cbb674f7a77437a8934aa841153dc52a538ce0...,0x7fcc933f6042f840864886fa7f32e0397d3218eb,3190409687568797587549,3188.65496224063474887584805,3188672750,3188.67275,0x853d955acef822db058eb8505911ed77f175b99e,Frax,...,18,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USD Coin,USDC,1,6,0x9a834b70c07c81a9fcd6f22e842bf002fbffbe4d,Uniswap V3 Frax/USD Coin 0.01%,3180433.84687286358399737442935,Frax/USD Coin


In [87]:
df_swaps.groupby("hash").hash.count().sort_values(ascending = False) # up to 7 swaps per a transaction

hash
0x8c6a57c4e449f5891b7a782fdb0762409c572cb5656ccd306095342f810f14cc    7
0x5371d5d9a8bde61b97d9546e902928cafbe710fafd32e8820bb1157533b99fda    7
0x1ae69f336a58f923b150bcea8428ec0efa321d948f07aa6a5ddde0905bc355cc    5
0xa7673197eabcfb07916097d8184412b540efde94d39374f640d9c1c95643cc0e    5
0xd0635a3f42031dfecbcd157772e6ed8abe977d050bbb541cd4dba422be588094    5
                                                                     ..
0x5e1224c026d584687c4b9797e1e10f81c302f520d43bd64082ed121ecc1be97b    1
0x5e1ed0ba826564f7e3cb89a6bb08c13303c4f0a9f331c537fce623c28ed327b7    1
0x5e3a7e36ee4625f9dbecf5a44afe486a6bc4f0b544723688e1f7685c9e3b774e    1
0x5e4a60d9eee44f8512c86f849267789214281e3bd9ad478a6f696aafd3939af4    1
0xfff2d9bf3582698da76762c5c0ba75130749799e0e45881bc467b0fe22c73dbf    1
Name: hash, Length: 4792, dtype: int64

In [102]:
df_swaps.groupby("tokenOut.symbol").amountOutUSD.sum().sort_values(ascending = False)

tokenOut.symbol
XSGD      996.9849414078393829575008475.2270568044003661...
PET       992.26005424357386567601849311564597446.334090...
MIDAS                             99.6967765201142772478386
ELON      98.9476319255237935432336613283027922.55632991...
FUN       965.54657274694462665772509174424731619.220970...
                                ...                        
 SHOP                                                     0
MAI                                                       0
M3M3                                                      0
BTRFLY                                                    0
PAYT                                                      0
Name: amountOutUSD, Length: 322, dtype: object

In [97]:
df_swaps[df_swaps["tokenOut.symbol"] == "XSGD"]

,timestamp,blockNumber,hash,from,amountIn,amountInUSD,amountOut,amountOutUSD,tokenIn.id,tokenIn.name,...,tokenIn.decimals,tokenOut.id,tokenOut.name,tokenOut.symbol,tokenOut.lastPriceUSD,tokenOut.decimals,pool.id,pool.name,pool.totalValueLockedUSD,pool.symbol
123,1676109587,16604782,0xb644dc3ef734010d823a9b8b0a1151d30e0cbbbafdea...,0xcf2e672d4af446293c65181e268e40ddcc270c30,997500000,997.5,1326488437,996.98494140783938295750084,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USD Coin,...,6,0x70e8de73ce538da2beed35d14187f6959a8eca96,XSGD,XSGD,0.7516217485064057,6,0x6279653c28f138c8b31b8a0f6f8cd2c58e8c1705,Uniswap V3 XSGD/USD Coin 0.05%,6256699.5061257577115016204806,XSGD/USD Coin
449,1676109047,16604737,0x263b115b050610bdbdbee51ad858ebcfeecedec2216b...,0x5505e2e2b1ca22c537cc3919540c4a7e08231249,49562500000000000,75.41518417963746892755147742302288,100089597,75.22705680440036619855204,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,Wrapped Ether,...,18,0x70e8de73ce538da2beed35d14187f6959a8eca96,XSGD,XSGD,0.7516217485064057,6,0xfca9090d2c91e11cc546b0d7e4918c79e0088194,Uniswap V3 XSGD/Wrapped Ether 0.05%,1469217.88897939940855366995834442,XSGD/Wrapped Ether
3907,1676102327,16604179,0x44aa734f2a2a33f38b8aec8fd4eff5d05e35af060a47...,0x42928111f6aac60d4be9bbaaec440229aab5b88e,225000000,225,299062876,224.88666997713248504609976,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USD Coin,...,6,0x70e8de73ce538da2beed35d14187f6959a8eca96,XSGD,XSGD,0.7516217485064057,6,0x6279653c28f138c8b31b8a0f6f8cd2c58e8c1705,Uniswap V3 XSGD/USD Coin 0.05%,6256699.5061257577115016204806,XSGD/USD Coin
4227,1676101799,16604135,0x561080937275e53b23827b4cf084c68730a6cf24b8de...,0x10763988715993eaf59905029ce3414ada0b5200,3423266042,3423.266042,4550369007,3421.36237922863218907926324,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USD Coin,...,6,0x70e8de73ce538da2beed35d14187f6959a8eca96,XSGD,XSGD,0.7516217485064057,6,0x6279653c28f138c8b31b8a0f6f8cd2c58e8c1705,Uniswap V3 XSGD/USD Coin 0.05%,6256699.5061257577115016204806,XSGD/USD Coin


In [105]:
df_swaps.groupby("tokenOut.symbol").hash.count().sort_values(ascending = False)

tokenOut.symbol
WETH     2347
USDC      906
USDT      365
HEX       230
WBTC       76
         ... 
NABOX       1
COW         1
SWEAT       1
SURE        1
INNBC       1
Name: hash, Length: 322, dtype: int64